In [99]:
import data_extract as de
import time
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

In [100]:
d = de.ACDB().get_temp_forecasts()

In [101]:
data = d[d['FCID'].isin(["North"])]

data = data[data['Value']<40]

data['month'] = data['StartTime'].dt.month

data['year'] = data['StartTime'].dt.year

data = data[data['month'].isin([1,2,11,12])]

data = data.rename(columns={'Value':"temp"})


In [102]:
a = de.ACDB().get_ERCOT_load_actuals()

In [103]:

load = a[a['date'].isin(data['StartTime'].to_list())]
load = load[load['FCID'].isin(['North'])]
load = load.rename(columns = {'Value':'load'})

In [104]:
b = de.ACDB().get_DAM_hub_spp()

In [105]:
dam = b[b['SettlementPointName'].isin(['HB_NORTH'])]

dam = dam[dam['StartTime'].isin(load['date'].to_list())]

dam = dam.rename(columns = {"Value":'DA'})

In [106]:
c = de.ACDB().get_RT_hub_spp()

c

,PCID,Value,StartTime,SettlementPointName,Interval
0,948,15.48,2016-01-01 00:00:00,HB_HOUSTON,15
1,948,15.33,2016-01-01 00:15:00,HB_HOUSTON,15
2,948,15.5,2016-01-01 00:30:00,HB_HOUSTON,15
3,948,15.43,2016-01-01 00:45:00,HB_HOUSTON,15
4,948,15.27,2016-01-01 01:00:00,HB_HOUSTON,15
...,...,...,...,...,...
1616875,953,12.68,2023-01-20 00:30:00,HB_WEST,15
1616876,953,12.79,2023-01-20 00:45:00,HB_WEST,15
1616877,953,12.92,2023-01-20 01:00:00,HB_WEST,15
1616878,953,12.83,2023-01-20 01:15:00,HB_WEST,15


In [107]:
rt = c[c['SettlementPointName'].isin(['HB_NORTH'])]

rt = rt[rt['StartTime'].isin(load['date'].to_list())]

rt = rt.rename(columns = {"Value":'rt'})

In [108]:
#join

m1 = pd.merge(data,load,left_on ="StartTime", right_on ="date")
m2 = pd.merge(m1,dam,left_on ="date", right_on ="StartTime")
m3 = pd.merge(m2,rt,left_on ="date", right_on ="StartTime")


In [112]:
final_data = m3[['FCID_x','date','temp','load','DA','rt']]

In [113]:
final_data

,FCID_x,date,temp,load,DA,rt
0,North,2017-12-31 00:00:00,31.5,1233.95,19.46,19.86
1,North,2017-12-31 00:00:00,31.5,1233.95,19.46,19.86
2,North,2017-12-31 00:00:00,31.5,1233.95,19.51,19.86
3,North,2017-12-31 00:00:00,31.5,1233.95,19.51,19.86
4,North,2017-12-31 00:00:00,31.5,1496.46,19.46,19.86
...,...,...,...,...,...,...
353891,West,2022-12-31 05:00:00,39.8,812.32,12.34,1.37
353892,West,2022-12-31 05:00:00,39.8,1403.22,12.69,-0.5
353893,West,2022-12-31 05:00:00,39.8,1403.22,12.69,1.37
353894,West,2022-12-31 05:00:00,39.8,1403.22,12.34,-0.5
